# Pie in The Sky: Food Syle Transfer with Cycle-GAN 

This file contains the ***training*** code of project of [ECE 285f](https://www.math.u-bordeaux.fr/~cdeledal/teaching.php#learningFA18) 2018.
We named this project as `Pie in The Sky` though it is not purely about food generation. But it can generate one type of food from the other, so it makes some `Pie in The Sky`. Actually, we truely get a food generator during the training of model. 


# Prerequisites
- Python 3.3 or above
- [Pytorch 0.4.0](torch.org)
- [Torchvision](https://github.com/pytorch/vision)
- [visdom visualizer 0.1.8.5](https://github.com/facebookresearch/visdom)
- [dominate 2.3.5](https://github.com/Knio/dominate)

For UCSD `DSMLP` user, you only need to install part of the package with the following command:
```bash
pip3 install visdom==0.1.8.5 --user
pip3 install dominate==2.3.5 --user
pip3 install torchvision --user
```

# Quick Start
In this project, we adopt [visdom](https://github.com/facebookresearch/visdom) for realtime visilization, which enable users to monitor the training process just in thier laptop, without cope another from the server.
So first, launch the visdom server. `Otherwise, the code will raise error!`
```bash
python3 -m visdom.server
```

Or, run it in background
```bash
nohup python3 -m visdom.server
```

# Babysitting your training
Since we have output the training visualization with visdom server to `port:8097`, you can observe the training by your browser. 
Visit `your_server_IP:8097` in your laptop, where `your_server_IP` is the IP address of your server. No password is needed.

## Import Note
- Before you training, please make sure that you have already launched visdom server, otherwise the training process will ***fail***.

## Author
Yuzhe Qin, Sihan Wang, Hang Zhang, Yawen Zhao, Leyan Zhu

In [ ]:
# Package Import
import time
from data import CreateDataLoader
from cycle_gan_model import CycleGANModel
from util.visualizer import Visualizer
import config

In [ ]:
# Load data and model
data_loader = CreateDataLoader(config)
dataset = data_loader.load_data()
dataset_size = len(data_loader)
print('***************************************************')
print('ECE 285f Project: Food Style Transfer')
print('Training demo start')
print('Training set size= %d' % dataset_size)

model = CycleGANModel()
model.initialize(config)
model.setup(config)
visualizer = Visualizer(config)
total_steps = 0

In [ ]:
# Training
for epoch in range(config.epoch_count, config.niter + config.niter_decay + 1):
    epoch_start_time = time.time()
    iter_data_time = time.time()
    epoch_iter = 0

    for i, data in enumerate(dataset):
        iter_start_time = time.time()
        if total_steps % 100 == 0:
            t_data = iter_start_time - iter_data_time
        visualizer.reset()
        total_steps += config.batch_size
        epoch_iter += config.batch_size
        model.set_input(data)
        model.optimize_parameters()

        # Visualization
        if total_steps % 400 == 0:
            save_result = total_steps % config.update_html_freq == 0
            visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

        # Print log
        if total_steps % 100 == 0:
            losses = model.get_current_losses()
            t = (time.time() - iter_start_time) / config.batch_size
            visualizer.print_current_losses(epoch, epoch_iter, losses, t, t_data)
            if config.display_id > 0:
                visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, config, losses)

        if total_steps % config.save_latest_freq == 0:
            print('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_steps))
            save_suffix = 'iter_%d' % total_steps if config.save_by_iter else 'latest'
            model.save_networks(save_suffix)

        iter_data_time = time.time()
        if epoch % 50 == 0:
            print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))
            model.save_networks('latest')
            model.save_networks(epoch)

        print('End of epoch %d / %d \t Time Taken: %d sec' %
              (epoch, config.niter + config.niter_decay, time.time() - epoch_start_time))
        model.update_learning_rate()